In [1]:
import os
import numpy as np
import pandas as pd
import pickle

import pyensembl

pd.set_option('display.max_columns', None)
os.chdir("/home/ylee/blast/blastdb/")

In [2]:
genome = pyensembl.EnsemblRelease(100, "mus_musculus")

In [3]:
# Load BLAST results into a pandas DataFrame
blast_Actg1 = pd.read_csv("blast_Actg1_local_wss7.table", sep="\t", comment='#', header=None)
# blast_Actg1 = pd.read_csv("blast_Actg1_local_ws4_eval5_1.table", sep="\t", comment='#', header=None)
# Define column names
# ["query id", "subject id", "% identity", "alignment length", "q. start", "q. end", "query length", "s. start", "s. end", "subject strand", "subject length", "mismatches", "gaps", "evalue", "bit score"]
blast_Actg1.columns = ["qseqid", "sseqid", "pident", "length", "qstart", "qend", "qlen", 
                                 "sstart", "send", "sstrand", "slen", "mismatch", "gaps", "evalue", "bitscore"]

blast_Actg1['genes'] = pd.NA
blast_Actg1['extend'] = pd.NA

blast_Actg1 = blast_Actg1.head(270)

In [4]:
blast_Actg1

,qseqid,sseqid,pident,length,qstart,qend,qlen,sstart,send,sstrand,slen,mismatch,gaps,evalue,bitscore,genes,extend
0,ENSMUST00000071555.12,chr4,99.738,1905,48,1952,1952,103563325,103565229,plus,156508116,5,0,0.000000,3491.0,NaN,NaN
1,ENSMUST00000071555.12,chr4,77.778,99,1120,1216,1952,56740228,56740133,minus,156508116,17,5,0.000051,56.5,NaN,NaN
2,ENSMUST00000071555.12,chr4,94.444,36,754,789,1952,154667036,154667001,minus,156508116,2,0,0.000051,56.5,NaN,NaN
3,ENSMUST00000071555.12,chr4,72.671,161,350,505,1952,154667440,154667285,minus,156508116,34,10,0.110000,45.4,NaN,NaN
4,ENSMUST00000071555.12,chr4,78.125,64,582,645,1952,154667208,154667145,minus,156508116,14,0,1.400000,41.7,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,ENSMUST00000071555.12,chr3,72.680,194,1703,1893,1952,83390771,83390583,minus,160039680,45,8,0.000014,58.4,NaN,NaN
266,ENSMUST00000071555.12,chr3,84.615,52,1902,1952,1952,83390072,83390022,minus,160039680,6,2,0.002000,51.0,NaN,NaN
267,ENSMUST00000071555.12,chr3,78.689,61,1136,1196,1952,30597950,30597890,minus,160039680,13,0,1.400000,41.7,NaN,NaN
268,ENSMUST00000071555.12,chr3,85.000,40,1805,1843,1952,123326807,123326845,plus,160039680,4,2,5.100000,39.9,NaN,NaN


## Function

In [5]:
# function that return gene IDs in locus
def find_gene_in_locus(chrNum, start, end, strand):
    gene_ids_at_locus = genome.gene_ids_at_locus(contig=chrNum, position=start, end=end)   # genes_at_locus is also possible
    gene_ids_at_locus += genome.gene_ids_at_locus(contig=chrNum, position=start)
    gene_ids_at_locus += genome.gene_ids_at_locus(contig=chrNum, position=end)
    
    return gene_ids_at_locus

In [6]:
chromosome_sizes = {
    "1": 195471971,
    "2": 182113224,
    "3": 160039680,
    "4": 156508116,
    "5": 151834684,
    "6": 149736546,
    "7": 145441459,
    "8": 129401213,
    "9": 124595110,
    "10": 130694993,
    "11": 122082543,
    "12": 120129022,
    "13": 120421639,
    "14": 124902244,
    "15": 104043685,
    "16": 98207768,
    "17": 94987271,
    "18": 90702639,
    "19": 61431566,
    "X": 171031299,
    "Y": 91744698,
    "MT": 16299
}

In [7]:
# function that extends hit locus
def extend_hit(chrNum, start, end, strand, extendLength = 1000):
    chr_length = chromosome_sizes[chrNum]                              
    move_start = start - extendLength
    move_end = end + extendLength
    annotation = "both"

    if move_start < 1 and move_end > chr_length:
        return [1, chr_length, "last extension"]
    else:
        if move_start < 1: 
            move_start = 1
            annotation = "upstream"
        elif move_end > chr_length: 
            move_end = chr_length
            annotation = "downstream"
            
    return [move_start, move_end, annotation]

In [8]:
# gene의 TSS, and distance
def check_distance(gene_id, hit_locus):
    
    # create locus from gene_id
    gene = genome.gene_by_id(gene_id)
    gene_locus = pyensembl.Locus(contig=gene.contig, start=gene.start, end=gene.end, strand=gene.strand)
    if gene.strand == "+":
        TSS_point = gene.start
    else:    
        TSS_point = gene.end

    # calculate the distance
    # distance = hit_locus.distance_to_locus(gene_locus)              # if strand is different, (+/-) they calculate distance as infinity.
    if hit_locus.distance_to_interval(gene.start, gene.end) == 0:
        distance = 0
    else:
        distance = hit_locus.distance_to_interval(TSS_point, TSS_point)
    
    return distance

## Main - add 2kb distance at hit locus

In [27]:
similar_pcg_set = set()
sigupregul_pcg_rows = []
appended_indices = {
    "ENSMUSG00000020246": set(),
    "ENSMUSG00000023868": set(),
    "ENSMUSG00000035783": set(),
    "ENSMUSG00000037351": set(),
    "ENSMUSG00000047497": set(),
    "ENSMUSG00000059430": set(),
    "ENSMUSG00000059895": set()
}

i = 0
while i < len(blast_Actg1): 
    row = blast_Actg1.iloc[i]

    hit_chrNum = row['sseqid'][3:]
    if row['sstrand']=='plus':
        hit_start = int(row['sstart'])
        hit_end =  int(row['send'])
        hit_strand = "+" 
    else:
        hit_start = int(row['send'])
        hit_end =  int(row['sstart'])
        hit_strand = "-"

    # create hit Locus object
    hit_locus = pyensembl.Locus(contig=hit_chrNum, start=hit_start, end=hit_end, strand=hit_strand)

    extend = "NA"
    while (extend != "false"):
        extend_hit_result = extend_hit(hit_chrNum, hit_start, hit_end, hit_strand, 2000)    # default extension
        hit_start = extend_hit_result[0]
        hit_end = extend_hit_result[1]
        gene_in_locus_list = find_gene_in_locus(hit_chrNum, hit_start, hit_end, hit_strand)             # find gene in locus
        if len(gene_in_locus_list) == 0:                                                                # if there is no gene in locus, extend hit length
            extend_hit_result = extend_hit(hit_chrNum, hit_start, hit_end, hit_strand, 1000)
            hit_start = extend_hit_result[0]
            hit_end = extend_hit_result[1]
            extend = extend_hit_result[2]
        else:                                                                                            # if there is gene in locus, print the gene id and distance
            # check distance between hit locus and gene
            info = ""
            for gene_id in gene_in_locus_list: 
                distance = check_distance(gene_id, hit_locus)
                gene = genome.gene_by_id(gene_id);
                info += f"{gene_id} - Distance: {distance}; "
                ####################################
                # print(gene_id, distance)    
                # if (gene_id == 'ENSMUSG00000070980'):
                #    print("Check ENSMUSG00000096930 distance : ", check_distance("ENSMUSG00000096930", hit_locus), "\n",
                #          "Hit start/end : ", hit_locus.start, hit_locus.end, "\n",
                #          "Extended Hit start/end : ", hit_start, hit_end, "\n",
                #          "Gene strand/start/end : ", genome.gene_by_id('ENSMUSG00000096930').strand, genome.gene_by_id('ENSMUSG00000096930').start, genome.gene_by_id('ENSMUSG00000096930').end)
                ####################################
                if distance < 2000 and gene.biotype == "protein_coding":
                    similar_pcg_set.add(gene_id)
                    
                    #################################
                    if gene.gene_id == "ENSMUSG00000020246":  # Hcfc2
                        if i not in appended_indices["ENSMUSG00000020246"]:
                            row_copy = row.copy()
                            row_copy['gene_id'] = "ENSMUSG00000020246"
                            row_copy['gene_name'] = "Hcfc2"
                            sigupregul_pcg_rows.append(row_copy)
                            appended_indices["ENSMUSG00000020246"].add(i)
                    
                    if gene.gene_id == "ENSMUSG00000023868":  # Pde10a
                        if i not in appended_indices["ENSMUSG00000023868"]:
                            row_copy = row.copy()
                            row_copy['gene_id'] = "ENSMUSG00000023868"
                            row_copy['gene_name'] = "Pde10a"
                            sigupregul_pcg_rows.append(row_copy)
                            appended_indices["ENSMUSG00000023868"].add(i)

                    if gene.gene_id == "ENSMUSG00000035783":  # Acta2
                        if i not in appended_indices["ENSMUSG00000035783"]:
                            row_copy = row.copy()
                            row_copy['gene_id'] = "ENSMUSG00000035783"
                            row_copy['gene_name'] = "Acta2"
                            sigupregul_pcg_rows.append(row_copy)
                            appended_indices["ENSMUSG00000035783"].add(i)
                    
                    if gene.gene_id == "ENSMUSG00000037351":  # Actr1b
                        if i not in appended_indices["ENSMUSG00000037351"]:
                            row_copy = row.copy()
                            row_copy['gene_id'] = "ENSMUSG00000037351"
                            row_copy['gene_name'] = "Actr1b"
                            sigupregul_pcg_rows.append(row_copy)
                            appended_indices["ENSMUSG00000037351"].add(i)
                    
                    if gene.gene_id == "ENSMUSG00000047497":  # Adamts12
                        if i not in appended_indices["ENSMUSG00000047497"]:
                            row_copy = row.copy()
                            row_copy['gene_id'] = "ENSMUSG00000047497"
                            row_copy['gene_name'] = "Adamts12"
                            sigupregul_pcg_rows.append(row_copy)
                            appended_indices["ENSMUSG00000047497"].add(i)

                    if gene.gene_id == "ENSMUSG00000059430":  # Actg2
                        if i not in appended_indices["ENSMUSG00000059430"]:
                            row_copy = row.copy()
                            row_copy['gene_id'] = "ENSMUSG00000059430"
                            row_copy['gene_name'] = "Actg2"
                            sigupregul_pcg_rows.append(row_copy)
                            appended_indices["ENSMUSG00000059430"].add(i)

                    if gene.gene_id == "ENSMUSG00000059895":  # Ptp4a3
                        if i not in appended_indices["ENSMUSG00000059895"]:
                            row_copy = row.copy()
                            row_copy['gene_id'] = "ENSMUSG00000059895"
                            row_copy['gene_name'] = "Ptp4a3"
                            sigupregul_pcg_rows.append(row_copy)
                            appended_indices["ENSMUSG00000059895"].add(i)
                    #################################
            
            # Update DataFrame, add gene and extend information
            # blast_Actg1.at[i, 'genes'] = info
            # blast_Actg1.at[i, 'extend'] = extend
            extend = 'false'

    i += 1

print(len(similar_pcg_set))

48


In [30]:
print(len(sigupregul_pcg_rows))

24


In [31]:
df_sigupregul_pcg = pd.DataFrame(sigupregul_pcg_rows)
df_sigupregul_pcg.to_csv('sigupregul_pcg_rows.csv', index=False)

In [53]:
os.chdir("/home/ylee/blast/result/")
with open('similar_pcg_set_Y.pkl', 'rb') as file:
    similar_pcg_set_Y = pickle.load(file)

with open('similar_pcg_set_C.pkl', 'rb') as file:
    similar_pcg_set_C = pickle.load(file)

similar_pcg_set_Common = similar_pcg_set_C - (similar_pcg_set_C - similar_pcg_set)
print(len(similar_pcg_set_Common))

with open('similar_pcg_set_Common.pkl', 'wb') as file:
    pickle.dump(similar_pcg_set_Common, file)

40


In [13]:
os.chdir("/home/ylee/blast/result/")

pcg_ids = [gene.gene_id for gene in genome.genes() if gene.biotype == 'protein_coding']
pcg_set = set(pcg_ids)
nonsimilar_pcg_set = pcg_set - similar_pcg_set

# Save the set as a pickle file
with open('similar_pcg_set_Y.pkl', 'wb') as file:
    pickle.dump(similar_pcg_set, file)

with open('nonsimilar_pcg_set_Y.pkl', 'wb') as file:
    pickle.dump(nonsimilar_pcg_set, file)

with open('pcg_set.pkl', 'wb') as file:
    pickle.dump(pcg_set, file)

## Codes from K

In [29]:
def check_overlapped(start_1, end_1, start_2, end_2):
    return max(start_1, start_2) <= min(end_1, end_2)

def check_TSS_below_2kb(gene, start, end):
    if gene.strand == '-':
        TSS = gene.end
    else:
        TSS = gene.start
        
    if start <= TSS <= end:
        distance =  0
    elif TSS < start:
        distance = start - TSS
    else:
        distance = TSS - end

    return distance <= 2000

In [101]:
similar_pcg_set_K = set()

i = 0
while i < len(blast_Actg1_local_ws4): 
    row = blast_Actg1_local_ws4.iloc[i]

    hit_chrNum = row['sseqid'][3:]
    if row['sstrand']=='plus':
        hit_start = int(row['sstart'])
        hit_end =  int(row['send'])
        hit_strand = "+" 
    else:
        hit_start = int(row['send'])
        hit_end =  int(row['sstart'])
        hit_strand = "-"

    # create hit Locus object
    hit_locus = pyensembl.Locus(contig=hit_chrNum, start=hit_start, end=hit_end, strand=hit_strand)

    extend = "NA"
    while (extend != "false"):
        extend_hit_result = extend_hit(hit_chrNum, hit_start, hit_end, hit_strand, 2000)    # default extension
        hit_start = extend_hit_result[0]
        hit_end = extend_hit_result[1]
        gene_in_locus_list = find_gene_in_locus(hit_chrNum, hit_start, hit_end, hit_strand)             # find gene in locus
        if len(gene_in_locus_list) == 0:                                                                # if there is no gene in locus, extend hit length
            extend_hit_result = extend_hit(hit_chrNum, hit_start, hit_end, hit_strand, 1000)
            hit_start = extend_hit_result[0]
            hit_end = extend_hit_result[1]
            extend = extend_hit_result[2]
        else:
            for gene_id in gene_in_locus_list: 
                gene = genome.gene_by_id(gene_id);
                check1 = check_overlapped(gene.start, gene.end, hit_locus.start, hit_locus.end);
                check2 = check_TSS_below_2kb(gene, hit_locus.start, hit_locus.end);
                if check1 or check2:
                    if gene.biotype == "protein_coding":
                        similar_pcg_set_K.add(gene_id)
            extend = 'false'

    i += 1

print(len(similar_pcg_set_K))

48
